In [1]:
%matplotlib inline
import os
from glob import glob
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#import censusgeocode as cg
import geopandas as gpd

In [2]:
data_dir = '/cluster/tufts/hugheslab/datasets/NSF_OD/'
result_dir = os.path.join(data_dir, 'results')
mass_shapefile = os.path.join(data_dir,'shapefiles','MA_2021')

if not os.path.exists(result_dir):
    os.makedirs(result_dir)

In [3]:

# not every file is an xlsx, 🤌
pre_2017_wildcard = os.path.join(data_dir,'MAOpiDths20*_clean.xls*')
post_2017_wildcard = os.path.join(data_dir, 'RVRS_Opioids*')

pre_2017_files = glob(pre_2017_wildcard)
pre_2017_years = [int(os.path.basename(file).split('_')[0][-4:]) for file in pre_2017_files]

post_2017_files = glob(post_2017_wildcard)
post_2017_years = [int(os.path.splitext(os.path.basename(file))[0].split('_')[2]) for file in post_2017_files]

all_files = pre_2017_files + post_2017_files
all_years = pre_2017_years + post_2017_years

relevant_files = []
relevant_years = []
for file, year in zip(all_files, all_years):
    filename = os.path.basename(file)
    
    
    # We have two files for 2020 & 2015, skip the first
    if (filename ==  'RVRS_Opioids_2020_20210129.xlsx') or \
       (filename == 'MAOpiDths2015_20190530_clean.xlsx'):
        continue
    relevant_files.append(file)
    relevant_years.append(year)
    
# Make sure years are unique
assert(len(set(relevant_years))==len(relevant_years))



In [4]:
address_df = pd.DataFrame()
all_filtered_df = pd.DataFrame()
for file, year in zip(relevant_files, relevant_years):
    
    single_year_df =  pd.read_excel(file, na_filter=False)
    
    if year < 2015:
        dod_col = "DOD"
        dod_format = "%Y%m%d"
    else:
        dod_col = "DOD_4_FD"
        dod_format = "%m/%d/%Y"
        
    
    

    # add year/quarter
    if year == 2014:
        missing_date = single_year_df['death_year']=='NA'
        num_miss = np.sum(missing_date)
        print(f'{num_miss} rows in 2014 dont have a death date, {num_miss/len(single_year_df)*100:.2f}% of total')
        single_year_df = single_year_df[~missing_date]
        single_year_df['dod_dt'] = pd.to_datetime({'year':single_year_df['death_year'],
                                                   'month': single_year_df['death_month'],
                                                   'day': single_year_df['death_day']})
    else:
        single_year_df['dod_dt'] = pd.to_datetime(single_year_df[dod_col], format=dod_format)
        
    single_year_df['year'] = single_year_df['dod_dt'].dt.year
    single_year_df['quarter'] = single_year_df['dod_dt'].dt.quarter

    has_ffix = year > 2014

    if has_ffix:
        address_cols = ['RES_ADDR_NUM', 'RES_STREET_PREFIX',
                    'RES_ADDR1', 'RES_STREET_DESIG',
                   'RES_STREET_SUFFIX']
        state = 'MASSACHUSETTS'
    else:
        address_cols = ['RES_ADDR_NUM',
                    'RES_ADDR1', 'RES_STREET_DESIG',
                   ]
        state = 'MA'

    if has_ffix and 'RES_STREET_PREFIX' not in single_year_df.columns:
        print(f'No decdent address column in {year}')
        #continue
        
    if year==2014:
        single_year_df[['RES_ADDR_NUM', 'RES_ADDR1', 'RES_CITY', 'RES_STATE']] = single_year_df['res_addres'].str.split(',', 3, expand=True)
        single_year_df.loc[:,['RES_STATE']] = single_year_df['RES_STATE'].str.strip()
        single_year_df[['RES_STREET_DESIG']] = ''
        single_year_df.loc[:,['RES_ZIP']] =  single_year_df['Postal'].apply(lambda x: '0'+str(x))
         
    tot_rows = single_year_df.shape[0]


    # remove unknown address
    filtered_df = single_year_df[single_year_df['RES_ADDR1'] != 'UNKNOWN']
    filtered_df = filtered_df[filtered_df['RES_ADDR1'] != 'UNK']
    try:
        count_filtered = single_year_df['RES_ADDR1'].value_counts()['UNKNOWN']
        count_filtered += single_year_df['RES_ADDR1'].value_counts()['UNK']
    except KeyError:
        count_filtered=0
    # remove blank address
    filtered_df = filtered_df[filtered_df['RES_ADDR1'] != '']
    try:
        count_filtered += single_year_df['RES_ADDR1'].value_counts()[''] 
    except KeyError:
        count_filtered += 0

    print(f'In {year} {count_filtered} rows have missing decdent address, '
          f'{count_filtered/tot_rows*100:.1f}% of total')

    # If street number is hyphenated, take first (123-125 -> 123)
    hyphenated_rows = (filtered_df['RES_ADDR_NUM'] != filtered_df['RES_ADDR_NUM'].apply(lambda x: x.split('-')[0])).sum()
    if hyphenated_rows >0 :
        print(f'Adjusting {hyphenated_rows} hyphenated addresses in {year}.')
    filtered_df.loc[:,'RES_ADDR_NUM'] =filtered_df['RES_ADDR_NUM'].apply(lambda x: x.split('-')[0])

    # Remove letters from street number
    alphabetic_rows = (filtered_df['RES_ADDR_NUM'] != filtered_df['RES_ADDR_NUM'].str.replace(r'\D','')).sum()
    if alphabetic_rows > 0:
        print(f'Adjusting {alphabetic_rows} addresses with letters in {year}.')

    filtered_df.loc[:,'RES_ADDR_NUM'] = filtered_df['RES_ADDR_NUM'].str.replace(r'\D','')



    filtered_df.loc[:,'address'] = filtered_df[address_cols].agg(' '.join, axis=1)

    if 'SFN_NUM' not in filtered_df.columns:
        print(f'{year} is missing SFN_NUM, creating new column [year]_[row]') 
        filtered_df['SFN_NUM'] = f'{year}_' + filtered_df.index.astype(str)
        
    count_other_states = filtered_df[filtered_df['RES_STATE']!=state].shape[0]
    print(f'Ignoring {count_other_states} decdencts not from  {state}')
    filtered_df = filtered_df[filtered_df['RES_STATE']==state]

    address_df = address_df.append(filtered_df[['SFN_NUM', 'address','RES_CITY', 'RES_STATE', 'RES_ZIP']])
    all_filtered_df = all_filtered_df.append(filtered_df)
    
    

XLRDError: Excel xlsx file; not supported

In [46]:
address_file1 = os.path.join(result_dir,'decedent_addresses_1.csv')
address_df.iloc[:7000,:].to_csv(address_file1, index=False)
address_file2 = os.path.join(result_dir,'decedent_addresses_2.csv')
address_df.iloc[7000:14000,:].to_csv(address_file2, index=False)
address_file3 = os.path.join(result_dir,'decedent_addresses_3.csv')
address_df.iloc[14000:,:].to_csv(address_file3, index=False)

In [47]:
start = time.time()

for pt, address_file in enumerate([address_file1,
                                   address_file2,
                                   address_file3]):
    response = cg.addressbatch(address_file)
    response_df = pd.DataFrame(response)
    response_df.to_csv(os.path.join(result_dir,f'res_response_pt{pt}.csv'),index=False)
    curr = time.time()
    print(f'Elapsed: {curr-start}')
    
    

Elapsed: 140.35547924041748
Elapsed: 232.1595458984375
Elapsed: 354.6980381011963


In [48]:
response_1 = pd.read_csv(os.path.join(result_dir,'res_response_pt0.csv'))
response_2 = pd.read_csv(os.path.join(result_dir,'res_response_pt1.csv'))
response_3 = pd.read_csv(os.path.join(result_dir,'res_response_pt2.csv'))

response_df = response_1.append(response_2).append(response_3)
response_df.to_csv(os.path.join(result_dir,'res_response_2000_2020.csv'), index=False)

/tmp/ipykernel_30102/2890640470.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  response_df = response_1.append(response_2).append(response_3)


In [3]:
response_df = pd.read_csv(os.path.join(result_dir,'res_response_2000_2020.csv'))

In [4]:
matched_df_no_year = response_df[response_df['match']]
count_matched = matched_df_no_year.shape[0]
count_filtered = response_df.shape[0]
print(f'Matched {count_matched} rows, {count_matched/count_filtered*100:.2f}% of all filtered rows')

Matched 21744 rows, 94.76% of all filtered rows


In [8]:
# get year into response
matched_df_no_year.loc[:,'id'] = matched_df_no_year.loc[:,'id'].astype(str)
all_filtered_df.loc[:,'SFN_NUM'] = all_filtered_df.loc[:,'SFN_NUM'].astype(str)
matched_df = matched_df_no_year.merge(all_filtered_df[['SFN_NUM','year', 'quarter', 'dod_dt']], left_on='id', right_on='SFN_NUM')
assert (len(matched_df)==len(matched_df_no_year))

/tmp/ipykernel_3893/2051165932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df_no_year.loc[:,'id'] = matched_df_no_year.loc[:,'id'].astype(str)


In [9]:
len(matched_df_no_year)

21744

In [10]:
matched_df.to_csv(os.path.join(result_dir,'geocoded_deaths_2000_2020.csv'), index=False)

In [11]:
mass_gdf = gpd.read_file(mass_shapefile)
matched_df.loc[:,'tract'] = matched_df['tract'].astype(int)
mass_gdf.loc[:,'TRACTCE'] = mass_gdf['TRACTCE'].astype(int)

In [12]:
matched_df

,id,address,match,matchtype,parsed,tigerlineid,side,statefp,countyfp,tract,block,lat,lon,SFN_NUM,year,quarter,dod_dt
0,2007_605,"58 MYRTLE ST, NORWOOD, MA, 2062",True,Exact,"58 MYRTLE ST, NORWOOD, MA, 02062",87243536.0,L,25.0,21.0,413202,1012.0,42.201649,-71.202264,2007_605,2007,4,2007-12-15
1,2002_349,"646 BIRCH ST, FALL RIVER, MA, 2724",True,Exact,"646 BIRCH ST, FALL RIVER, MA, 02724",46767810.0,L,25.0,5.0,640400,2005.0,41.684234,-71.176381,2002_349,2002,3,2002-07-15
2,2007_607,"573 OSBORN ST, FALL RIVER, MA, 2724",True,Exact,"573 OSBORN ST, FALL RIVER, MA, 02724",651813019.0,R,25.0,5.0,640500,1005.0,41.690942,-71.160056,2007_607,2007,4,2007-12-16
3,2002_348,"8 MICHIGAN AV, LYNN, MA, 1902",True,Exact,"8 MICHIGAN AVE, LYNN, MA, 01902",86646827.0,L,25.0,9.0,206600,4000.0,42.467733,-70.924900,2002_348,2002,3,2002-07-17
4,2007_606,"175 OSBORNE ST, FALL RIVER, MA, 2724",True,Non_Exact,"175 OSBORN ST, FALL RIVER, MA, 02724",46770133.0,R,25.0,5.0,640500,3001.0,41.692127,-71.166843,2007_606,2007,4,2007-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21739,2017_1708,"61 NORWOOD STREET , FALL RIVER, MASSACHUSETTS...",True,Exact,"61 NORWOOD ST, FALL RIVER, MA, 02723",46771908.0,R,25.0,5.0,641600,1003.0,41.693014,-71.127945,2017_1708,2017,4,2017-12-21
21740,2017_1707,"863 WASHINGTON STREET , WHITMAN, MASSACHUSETT...",True,Exact,"863 WASHINGTON ST, WHITMAN, MA, 02382",87445852.0,R,25.0,23.0,521201,2004.0,42.087970,-70.941268,2017_1707,2017,4,2017-12-12
21741,2018_497,"53 POLLEY LANE , WALPOLE, MASSACHUSETTS, 02032",True,Non_Exact,"53 POLLEY LN, EAST WALPOLE, MA, 02032",87297383.0,R,25.0,21.0,411200,2001.0,42.153999,-71.220725,2018_497,2018,1,2018-03-30
21742,2018_498,"56 MUDGE STREET , LYNN, MASSACHUSETTS, 01902",True,Exact,"56 MUDGE ST, LYNN, MA, 01902",647434896.0,L,25.0,9.0,206400,2014.0,42.477214,-70.928066,2018_498,2018,1,2018-03-31


In [7]:
deaths_per_tract_df = matched_df.groupby(['year','quarter','tract']).size().reset_index(name='deaths')

NameError: name 'matched_df' is not defined

In [14]:
no = 0
for tract in deaths_per_tract_df.tract.unique():
    if tract not in mass_gdf.TRACTCE.unique():
        raise ValueError('Failed to match a tract!')

In [15]:
deaths_gdf = gpd.GeoDataFrame()
for year in deaths_per_tract_df.year.unique():
    for quarter in deaths_per_tract_df.quarter.unique():
        these_deaths = deaths_per_tract_df[(deaths_per_tract_df['year']==year) & (deaths_per_tract_df['quarter']==quarter)]
        years_merged_deaths = mass_gdf.merge(these_deaths,
                                             left_on='TRACTCE',
                                             right_on='tract',
                                             how='left')
        # fill NAs
        years_merged_deaths.loc[:,'year']=year
        years_merged_deaths.loc[:,'quarter']=quarter
        years_merged_deaths.loc[:,'deaths'] = years_merged_deaths.loc[:,'deaths'].fillna(0)
        deaths_gdf = deaths_gdf.append(years_merged_deaths)

/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will 

/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will 

/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deaths_gdf = deaths_gdf.append(years_merged_deaths)
/tmp/ipykernel_3893/1952240894.py:13: FutureWarning: The frame.append method is deprecated and will 

In [17]:
deaths_file = os.path.join(result_dir,'res_deaths_qtr_all')
deaths_gdf.to_file(deaths_file)
matched_file = os.path.join(result_dir,'geocoded_deaths.csv')
matched_df.to_csv(matched_file, index=False)

/cluster/tufts/hugheslab/kheuto01/miniconda3/envs/geopanas/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [16]:
deaths_gdf['quarter']

0       1
1       1
2       1
3       1
4       1
       ..
1615    4
1616    4
1617    4
1618    4
1619    4
Name: quarter, Length: 142560, dtype: int64